<font size=4, color=red>**Training_script-v1.0**</font>

- [Vision_encoder](#Vision_model)
- [Text_encoder](#Text_model)
- [Unet_model](#Unet_model)



**TODO Lists**  
- [ ] Running the whole procedure using random data  
- [ ] Baseline  
- [ ] Achieve the dataloader  
- [ ] Try to train and eval the result  

In [ ]:
# requirements
# !pip install clip-interrogator==0.5.4
# !pip install git+https://github.com/openai/CLIP.git


In [1]:
# !pip install -q --upgrade diffusers transformers accelerate
import yaml
import math
import os
import timm
import logging
from collections import OrderedDict
from typing import Tuple, Union
from dataclasses import dataclass, field
from typing import List

import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn

import datasets
import diffusers
import transformers
# from diffusers import UNet2DConditionModel

from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, CLIPVisionModel

from diffusers.configuration_utils import ConfigMixin
from diffusers.models.cross_attention import AttnProcessor
# from diffusers.models.embeddings import TimestepEmbedding, Timesteps


from model import UNet2DConditionModel

from torchinfo import summary
import open_clip

/root/miniconda3/lib/python3.8/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


## New_model

In [2]:
from model.clip import build_model

clip_model = torch.jit.load('/root/RN50.pt').eval()
backbone = build_model(clip_model.state_dict(), 77).float()

In [5]:
img = torch.randn(1,3,334,334)
text = torch.randn(1,77,1024).long()


In [16]:
??backbone.encode_image

Signature: backbone.encode_image(image)
Docstring: <no docstring>
Source:   
    def encode_image(self, image):
        return self.visual(image.type(self.dtype))
File:      ~/autodl-tmp/Gradudate_Enflame/model/clip.py
Type:      method


In [7]:
out_img = backbone.encode_image(img)
# out_text = backbone.encode_text(text)

In [15]:
out_img[0].shape
out_img[1].shape
out_img[2].shape

torch.Size([1, 1024, 10, 10])

In [ ]:
class DIFFusoin(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Visible && ir encoder
        clip_model = torch.jit.load(cfg.clip_pretrain,
                                   map_location="cpu").eval()

## Vision_model

In [2]:
# down_dir = '/root/.cache/clip/'
# open_clip.list_models()



In [3]:
# clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')

In [4]:
pixel_inp = torch.randn(1,3,336,336)
text_inp = torch.randn(1,8,768)

inp = [pixel_inp, text_inp]

In [5]:
clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14-336', pretrained=False)

In [6]:
## hook
hook_in = []
hook_out = []
def hook_hidden_state(module, inp, out):
    # print(f'module:{module}')
    hook_in.append(inp)
    hook_out.append(out)
    # print(xx)
clip_model.visual.transformer.register_forward_hook(hook_hidden_state)

In [7]:
out = clip_model.encode_image(pixel_inp)

hook_in[0][0].shape # torch.Size([577, 1, 1024])

hook_out[0].shape 

torch.Size([577, 1, 1024])

In [120]:
# from transformers import CLIPVisionModel, CLIPVisionConfig
# from transformers import AutoProcessor, CLIPModel, CLIPVisionModelWithProjection

# from PIL import Image
# import requests
# vision_config = {
#     'hidden_size': 768,
#     'intermediate_size': 3072,
#     'image_size': 224,
#     'patch_size': 32,
# }
# ## create vision model
# model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# inputs = processor(images=image, return_tensors="pt")

# outputs = model(**inputs)
# pixel_lhs = outputs.last_hidden_state ## [B,S,D]
# pooled_output = outputs.pooler_output  # pooled CLS states [B, D]

## New_model

In [ ]:
class Diffusion(nn.Mu)

## Text_model

In [132]:
# from transformers import CLIPTextModel, CLIPTextModelWithProjection
# from transformers import AutoProcessor, CLIPModel, AutoTokenizer
# import warnings

# warnings.filterwarnings('ignore')
# text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-base-patch32')

# tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")

# outputs = text_encoder(**inputs) 
# text_lhs = outputs.last_hidden_state # [B, S, D]
# pooled_output = outputs.pooler_output


In [17]:
text_hook_in = []
text_hook_out = []
def text_hidden_hook(module, inp, out):
    text_hook_in.append(inp)
    text_hook_out.append(out)

clip_model.transformer.resblocks[-1].register_forward_hook(text_hidden_hook)

In [14]:
tokenizer = open_clip.get_tokenizer('ViT-L-14-336')
text = tokenizer(["a diagram", "a dog", "a cat"])
clip_model.encode_text(text=text)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>                                                                                │
│                                                                                                  │
│   1 tokenizer = open_clip.get_tokenizer('ViT-L-14-336')                                          │
│   2 text = tokenizer(["a diagram", "a dog", "a cat"])                                            │
│ ❱ 3 clip_model.encode_text(text=text)                                                            │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'clip_model' is not defined

In [17]:
type(text[0])

torch.Tensor

## Unet_model

In [ ]:
# for k in list(unet_ckpt.keys()):
#     if k.split('.')[-2] in ['time_emb_proj', 'time_emb']:
#         unet_ckpt.pop(k)

In [23]:
unet = UNet2DConditionModel(cross_attention_dim=768)
unet_ckpt = torch.load('ckpt/unet_model_no_conv.bin')   
unet.load_state_dict(unet_ckpt, strict=False)

inp = torch.randn(1,3,64,64)
tx_state = torch.randn(1,77,768)

opt = unet(inp, tx_state)

In [59]:
list(unet.named_children())[0][1].bias.shape

torch.Size([320])

In [55]:
for m in unet.named_children():
    if m[0] in ['conv_in']:
        print(m[1])

Conv2d(3, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


## Total_model

In [2]:
@dataclass()
class CFG:
    unet_config={
        'cross_attention_dim': 768,
    }
    unet_pretrained='ckpt/unet_model_no_conv.bin'
    

In [66]:
class Unet_Attn_model(nn.Module):
    def __init__(self,
                config,
                 clip_name:str = None,
                 clip_pretrained = None,
                 vision_encoder = None,
                 text_encoder = None,
                 text_decoer = None,
                 tokenizer = None,
                ):
        assert clip_name or vision_encoder, 'You must pass a encoder!'
        '''
        Params:
            clip_name: CLIP-model's name. (eps. ViT-B/16)
            clip_pretrained: Weather using pretrained.
            vision_encoder: if needed, not compatible with clip_name&clip_pretrained.
            text_encoder: if needed, not compatible with clip_name&clip_pretrained.
            tokenizer: tokenizer's name.
            config: Extra added config.
        '''
        super().__init__()
    
        self.tokenizer = tokenizer
        self.text_decoer = text_encoder
        self.vision_encoder = vision_encoder
        self.text_decoer = text_decoer
        self.config = config
        
        ## create clip_model based on the clip_name
        __CLIP_NAME_LS__ = open_clip.list_models()
        if clip_name:
            assert clip_name in __CLIP_NAME_LS__, 'CLIP_name is not in supported model_hub'
            self.clip_model, _, self.preprocess = open_clip.create_model_and_transforms(clip_name, pretrained=clip_pretrained)
            
        ## register_forward_hook for clip_encode_visual to extract the visual_hidden_state before the last layer
        self.clip_model.visual.transformer.register_forward_hook(hook=self.hook_visual_hidden_state)
        self.clip_model.transformer.resblocks[-1].register_forward_hook(hook=self.hook_text_hidden_state)
        self.visual_hook_in = []
        self.visual_hook_out = []
        self.text_hook_in = []
        self.text_hook_out = []
        ## create encosder  if needed
        if vision_encoder and not clip_name:
            self.visual_encoder = VisionTransformer(
                img_size=config['image_res'], patch_size=16, embed_dim=768, depth=12, num_heads=12, 
                mlp_ratio=4, qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6)) 
        ## text_
        if tokenizer:
            self.tokenizer = open_clip.get_tokenizer(tokenizer)
        
        ## Froze the clip_encoder
        self.clip_model.requires_grad_(False)
        
        ## Unet
        cross_attention_dim = self.config.unet_config['cross_attention_dim']
        self.unet = UNet2DConditionModel(cross_attention_dim=cross_attention_dim)
        if self.config.unet_pretrained:
            self.unet.load_state_dict(torch.load(self.config.unet_pretrained), strict=False)
            
        ## init_conv_in/out
        self.init_params()
        
    def forward(self, pixel_inp, text_inp):
        # visual features
        img_level_out = self.clip_model.encode_image(pixel_inp)
        ## pick out the visual hidden_state
        pixel_hidden_state = self.visual_hook_out[0]

        # text features
        if isinstance(text_inp[0], torch.Tensor):
            text_ = text_
        elif isinstance(text_inp[0], str):
            text_ = self.tokenizer(text_inp)

        seq_level_out = self.clip_model.encode_text(text_)
        ## pick out text hidden_state
        text_hidden_state = self.text_hook_out[0]

        # passing text&visual hidden_state to unet
        out_ = self.unet(pixel_hidden_state, text_hidden_state)

        # return 
        return out_.sample
    
    @staticmethod
    def encode_image(self, pixel_inp):
        return self.clip_model.encode_image(pixel_inp)
    
    @staticmethod
    def encode_text(self, text_inp):
        return self.clip_model.encode_text(text_inp)
    
    def init_params(self):
        for m in self.unet.named_children():
            if m[0] in ['conv_in', 'conv_out']:
                torch.nn.init.kaiming_normal_(
                    m[1].weight.data,
                    mode='fan_in'
                )
                if m[1].bias is not None:
                    torch.nn.init.constant_(m[1].bias.data, 0)
                print(f"Init the params in conv_in and conv_out!")

    def UnPatchify(self,)
    def hook_visual_hidden_state(self, *inp):
        print(inp[1][0].shape)
        print(inp[2].shape)
        print(inp[1][0] == inp[2])
        self.visual_hook_in.append(inp[1][0])
        self.visual_hook_out.append(inp[2])


    def hook_text_hidden_state(self, *inp):
        self.text_hook_in.append(inp[1][0])
        self.text_hook_out.append(inp[2])

In [74]:
336 / 14

24.0

In [75]:
??model.clip_model.visual


Signature:       model.clip_model.visual(*input, **kwargs)
Type:            VisionTransformer
String form:    
VisionTransformer(
  (patchnorm_pre_ln): Identity()
  (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0): ResidualAttentionBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate=none)
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (ls_2): Identity(

In [67]:
config = CFG()
config.unet_config

{'cross_attention_dim': 768}

In [68]:
# test model
model = Unet_Attn_model(
    config=config,
    clip_name='ViT-L-14-336',
    tokenizer='ViT-L-14-336'
)

Init the params in conv_in and conv_out!
Init the params in conv_in and conv_out!


In [84]:
from einops import rearrange, repeat
patch_inp = torch.randn(577,1,1024)

In [ ]:
unpatch = rearrange(patch_inp, ('n b c' -> 'b c '))

In [82]:
patch_inp[1:, :, :].permute(1,2,0).reshape()

torch.Size([1, 576, 1024])

In [69]:
txt = ["a diagram", "a dog", "a cat"]
image = torch.randn(1,3,336,336)

In [70]:
out = model(image, txt)


torch.Size([577, 1, 1024])
torch.Size([577, 1, 1024])
tensor([[[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        ...,

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]]])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>                                                                                │
│                                                                                                  │
│ ❱ 1 out = model(image, txt)                                                                      │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ in forward                                                                                       │
│                                                                                                  │
│    77 │   │   text_hidden_state = self.text_hook_out[0]                                          │
│    78 │   │                                                                                      │
│    79 │   │   # passing text&visual hidden_state to unet                                         │
│ ❱  80 │   │   out_ = self.unet(pixel_hidden_state, text_hidden_state)                            │
│    81 │   │                                                                                      │
│    82 │   │   # return                                                                           │
│    83 │   │   return out_.sample                                                                 │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/intern/zihao/Gradudate_Enflame/model/unet_condition.py:459 in forward                      │
│                                                                                                  │
│   456 │   │   │   emb = emb + class_emb                                                          │
│   457 │   │                                                

## Train-Phrase

In [50]:
import argparse
import os
import ruamel_yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path

import utils
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torch.distributed as dist

# from models.model_vqa import ALBEF
# from models.vit import interpolate_pos_embed
# from models.tokenization_bert import BertTokenizer


# from dataset.utils import save_result
# from dataset import create_dataset, create_sampler, create_loader, vqa_collate_fn

from scheduler import create_scheduler
from optim import create_optimizer
## tensorboard
from torch.utils.tensorboard import SummaryWriter

In [ ]:
def train(model, data_loader, optimizer, tokenizer, epoch, warmup_steps, device, scheduler, config, Writer):

    # train
    model.train()  
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('loss', utils.SmoothedValue(window_size=1, fmt='{value:.4f}'))

    header = 'Train Epoch: [{}]'.format(epoch)
    print_freq = 50    
    step_size = 100
    warmup_iterations = warmup_steps*step_size  
    
    for i,(image, question, answer, weights, n) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        image, weights = image.to(device,non_blocking=True), weights.to(device,non_blocking=True)      
        question_input = tokenizer(question, padding='longest', truncation=True, max_length=25, return_tensors="pt").to(device) 
        answer_input = tokenizer(answer, padding='longest', return_tensors="pt").to(device) 
        
        if epoch>0 or not config['warm_up']:
            alpha = config['alpha']
        else:
            alpha = config['alpha']*min(1,i/len(data_loader))

        loss = model(image, question_input, answer_input, train=True, alpha=alpha, k=n, weights=weights)        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
     
        if epoch==0 and i%step_size==0 and i<=warmup_iterations: 
            scheduler.step(i//step_size) 
            
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger.global_avg())   
    ## Extra added
    if dist.get_rank() == 0:
        Writer.add_scalar('LOSS', getattr(metric_logger, 'loss'), i+epoch)
        Writer.add_scalar('lr', optimizer.param_groups[0]['lr'], i+epoch)
        
    return {k: "{:.3f}".format(meter.global_avg) for k, meter in metric_logger.meters.items()} 


@torch.no_grad()
def evaluation(model, data_loader, tokenizer, device, config) :
    # test
    model.eval()
            
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Generate VQA test result:'
    print_freq = 50
    
    result = []
    
    answer_list = [answer+config['eos'] for answer in data_loader.dataset.answer_list]
    answer_input = tokenizer(answer_list, padding='longest', return_tensors='pt').to(device)    
        
    for n, (image, question, question_id) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):        
        image = image.to(device,non_blocking=True)             
        question_input = tokenizer(question, padding='longest', return_tensors="pt").to(device)        

        topk_ids, topk_probs = model(image, question_input, answer_input, train=False, k=config['k_test'])      
        
        for ques_id, topk_id, topk_prob in zip(question_id, topk_ids, topk_probs):
            ques_id = int(ques_id.item())          
            _, pred = topk_prob.max(dim=0)
            result.append({"question_id":ques_id, "answer":data_loader.dataset.answer_list[topk_id[pred]]})   

    return result


def main(args, config):
    ## distribued_init
    utils.init_distributed_mode(args)    
    
    device = torch.device(args.device)
    ## Writer
    Writer = None
    if dist.get_rank ==0 :
        Writer = SummaryWriter(config['tensorboard_dir'])
        
    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.benchmark = True
    
    start_epoch = 0
    max_epoch = config['schedular']['epochs']
    warmup_steps = config['schedular']['warmup_epochs']
    
    
    #### Dataset #### 
    print("Creating vqa datasets")
    datasets = create_dataset('vqa', config)   
    
    if args.distributed:
        num_tasks = utils.get_world_size()
        global_rank = utils.get_rank()            
        samplers = create_sampler(datasets, [True, False], num_tasks, global_rank)         
    else:
        samplers = [None, None]
    
    train_loader, test_loader = create_loader(datasets,samplers,
                                              batch_size=[config['batch_size_train'],config['batch_size_test']],
                                              num_workers=[4,4],is_trains=[True, False], 
                                              collate_fns=[vqa_collate_fn,None]) 

    tokenizer = BertTokenizer.from_pretrained(args.text_encoder)

    #### Model #### 
    print("Creating model")
    model = ALBEF(config=config, text_encoder=args.text_encoder, text_decoder=args.text_decoder, tokenizer=tokenizer)
    model = model.to(device)   
    
    arg_opt = utils.AttrDict(config['optimizer'])
    optimizer = create_optimizer(arg_opt, model)
    arg_sche = utils.AttrDict(config['schedular'])
    lr_scheduler, _ = create_scheduler(arg_sche, optimizer)          
        
    if args.checkpoint:    
        checkpoint = torch.load(args.checkpoint, map_location='cpu') 
        state_dict = checkpoint['model']
        
        # reshape positional embedding to accomodate for image resolution change
        pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder.pos_embed'],model.visual_encoder)         
        state_dict['visual_encoder.pos_embed'] = pos_embed_reshaped   
        
        if not args.evaluate:
            if config['distill']:
                m_pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder_m.pos_embed'],model.visual_encoder_m)   
                state_dict['visual_encoder_m.pos_embed'] = m_pos_embed_reshaped 
                
            for key in list(state_dict.keys()):
                if 'bert' in key:
                    encoder_key = key.replace('bert.','')         
                    state_dict[encoder_key] = state_dict[key] 
                # intialize text decoder as multimodal encoder (last 6 layers of model.text_encoder)    
                if 'text_encoder' in key:                
                    if 'layer' in key:
                        encoder_keys = key.split('.')
                        layer_num = int(encoder_keys[4])
                        if layer_num<6:
                            del state_dict[key]  
                            continue
                        else:
                            decoder_layer_num = (layer_num-6)
                            encoder_keys[4] = str(decoder_layer_num)
                            encoder_key = '.'.join(encoder_keys)     
                    else:
                        encoder_key = key
                    decoder_key = encoder_key.replace('text_encoder','text_decoder')  
                    state_dict[decoder_key] = state_dict[key]     

                    del state_dict[key]                
                
        msg = model.load_state_dict(state_dict,strict=False)  
        print('load checkpoint from %s'%args.checkpoint)
        print(msg)  

        
    model_without_ddp = model
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        model_without_ddp = model.module    
    
    
    print("Start training")
    start_time = time.time()

    for epoch in range(start_epoch, max_epoch):
        if epoch>0:
            lr_scheduler.step(epoch+warmup_steps)  
        
        if not args.evaluate:
            if args.distributed:
                train_loader.sampler.set_epoch(epoch)

            train_stats = train(model, train_loader, optimizer, tokenizer, epoch, warmup_steps, device, lr_scheduler, config, Writer)  ## Add Writer

        if args.evaluate:
            break
            
        if utils.is_main_process():               
            log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                         'epoch': epoch,
                        }                
            with open(os.path.join(args.output_dir, "log.txt"),"a") as f:
                f.write(json.dumps(log_stats) + "\n")                        
                         
            save_obj = {
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'config': config,
                'epoch': epoch,
            }
            torch.save(save_obj, os.path.join(args.output_dir, 'checkpoint_%02d.pth'%epoch))  

        dist.barrier()   
  
    vqa_result = evaluation(model, test_loader, tokenizer, device, config)        
    result_file = save_result(vqa_result, args.result_dir, 'vqa_result_epoch%d'%epoch)
                     
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str)) 
    
            

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--config', default='./configs/VQA.yaml') 
    parser.add_argument('--checkpoint', default='') 
    parser.add_argument('--output_dir', default='output/vqa')
    parser.add_argument('--evaluate', action='store_true')    
    parser.add_argument('--text_encoder', default='bert-base-uncased')
    parser.add_argument('--text_decoder', default='bert-base-uncased')
    parser.add_argument('--device', default='cuda')
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--world_size', default=1, type=int, help='number of distributed processes')    
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    parser.add_argument('--distributed', default=True, type=bool)
    ## ADD
#     parser.add_argument('--tensorboard_dir', type=str, default='./tensenboard/')
    args = parser.parse_args()

    config = yaml.load(open(args.config, 'r'), Loader=yaml.Loader)

    args.result_dir = os.path.join(args.output_dir, 'result')

    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    Path(args.result_dir).mkdir(parents=True, exist_ok=True)
        
    yaml.dump(config, open(os.path.join(args.output_dir, 'config.yaml'), 'w'))
    
    main(args, config)

<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Install & Import all dependencies </b></div>

#### <a id="top"></a>
# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>
​
<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
​
* [1. About CLIP interrogator tool](#1)
* [2. Install & Import all dependencies](#2)
* [3. Set configs](#3)
* [4. Load the Sample Submission](#4)
* [5. Build index from images](#5)
* [6. Load the embedding model](#6)
* [7. Prepare CLIP interrogator tool](#7)
    * [7.1 Define CLIP interrogator config](#7.1)
    * [7.2 Define BLIP model](#7.2)
    * [7.3 Define CLIP model](#7.3)
    * [7.4 Create CLIP interrogator object](#7.4)
* [8. Define interrogate function](#8)
    * [8.1 Get labels embeddings](#8.1)
    * [8.2 Create main interrogation function](#8.2)
* [9. Extract promt from images](#9)
    * [9.1 Check the result](#9.1)
* [10. Create a sample submission with a constant prompt prediction](#10)
    * [10.1 Encode prompts](#10.1)
    * [10.2 Create submission DataFrame and save it as a .csv file](#10.2)